Import packages

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import pulp
import spopt
from spopt.locate import PMedian

from scripts import create_allocation_map

Import data

In [ ]:
_file_location = Path().resolve()

In [ ]:
students_df = pd.read_csv(_file_location / "data" / "example_subject_students.csv")
schools_df = pd.read_csv(_file_location / "data" / "example_subject_schools.csv")

Do pre-processing from <https://github.com/UCL/ioe-student-school-allocation/>.
Install using `pip`:
```python
python -m pip install --upgrade pip
python -m pip install -e .
```

You will also need to add a TfL API key, available from 
<https://api-portal.tfl.gov.uk/>. It is set in `.envrc_sample` `export TFL_APP_KEY=`
What you should do is 
```sh
cp .envrc_sample .envrc
```
Then put in the key. Then run
```sh
source .envrc
```
Then re-run. You can check if it’s worked by running
`echo $TFL_APP_KEY`, and `export N_CORES=1`.

Run using
```sh
tfl data example_subject
```

Running time
- 25 min on 1 core for 10 students, 70 schools, 3 failures
- 14 min on 4 cores for 20 students, 70 schools
- 12 min on 8 cores for 20 students, 70 schools, 10 failures
- 6 min on 16 cores for 19 students, 70 schools, 2 failures

Read in sample data

In [ ]:
example_subject_time = pd.read_csv(
    _file_location / "data" / "example_subject_student_school_journeys.csv"
)

A large value to fix optimisation

In [ ]:
LARGE_VALUE_PLACEHOLDER = 10_000

Create pivot table from data

In [ ]:
example_subject_time_table = (
    example_subject_time.pivot_table(
        columns="school",
        fill_value=LARGE_VALUE_PLACEHOLDER,
        index="student",
        sort=False,
        values="time",
    )
    .astype(int)
    .values
)

In [ ]:
print(example_subject_time_table)

Clean data for the model

Define the function to clean school and student dataframe
to only keep the students and schools which have successful journeys

In [ ]:
def data_clean(
    df: pd.DataFrame, id_col: str, time_col: str, time: pd.DataFrame
) -> pd.DataFrame:
    ids_to_remove = set(df[id_col]) - set(time[time_col])
    mask = ~df[id_col].isin(ids_to_remove)
    return df[mask].reset_index().drop("index", axis=1)

In [ ]:
schools_df_clean = data_clean(
    schools_df, "SE2 PP: Code", "school", example_subject_time
)
students_df_clean = data_clean(students_df, "ST: ID", "student", example_subject_time)

Check if data is okay

In [ ]:
assert len(schools_df_clean) == len(example_subject_time_table[0])
assert len(students_df_clean) == len(example_subject_time_table)

The version should be `0.1.dev971+gbc75cde` or similar

In [ ]:
print(spopt.__version__)

Data preparing

a. set the amount of each demand point: in IOE case, it is 1.

In [ ]:
demand = np.ones(len(students_df_clean))

b. Pick out predefined facilities: priority 1 schools
please notice that the column name of priority can vary, for maths it's 'MAT priority' etc.

In [ ]:
schools_priority_1 = schools_df_clean[
    schools_df_clean["MAT priority"] == 1
].index.tolist()
schools_priority_1_arr = np.array(schools_priority_1)

c. set the facility capacities

In [ ]:
capacities_arr = np.array(schools_df_clean["Count"])

Run the model


If you get this error:
>Problem is infeasible. The predefined facilities can't be 
>fulfilled, because their capacity is larger than the total 
>demand 10.0.
This is because you have more priority schools (priority = 1 or 2? need to check)
than you do students who need placements. You need more schools, or fewer students. 
                        
The `fulfill_predefined_fac` must be true, it is used to guarantee priority 1
schools will be fulfilled

In [ ]:
solver = pulp.PULP_CBC_CMD()
pmedian_from_cost_matrix = PMedian.from_cost_matrix(
    example_subject_time_table,
    demand,
    p_facilities=len(students_df_clean),
    predefined_facilities_arr=schools_priority_1_arr,
    facility_capacities=capacities_arr,
    fulfill_predefined_fac=True,
)
pmedian_from_cost_matrix = pmedian_from_cost_matrix.solve(solver)

Save the match result

In [ ]:
match_df = students_df_clean

for i in range(len(students_df_clean)):
    school_index = pmedian_from_cost_matrix.cli2fac[i]
    match_df.loc[i, "allocation_school_id"] = schools_df_clean.loc[
        school_index[0], "SE2 PP: Code"
    ]

In [ ]:
match_df.to_csv(_file_location / "data" / "example_subject_matches.csv")

Run the map creation py document

In [ ]:
create_allocation_map.main("example_subject")